In [ ]:
##Data Simulation

In [1]:
!pip install faker pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.9 MB/s eta 0:00:008.0 MB/s eta 0:00:01


In [ ]:
#Import and setup cell

In [2]:
import mysql.connector
from faker import Faker
from datetime import datetime, timedelta
import random

fake = Faker()

db_config = {
    "host": "localhost",
    "user": "python_user",
    "password": "harshada",
    "database": "security_analytics"
}

def get_connection():
    return mysql.connector.connect(**db_config)


In [3]:
##Insert Departments
departments = ["IT", "Finance", "HR", "Engineering"]

conn = get_connection()
cursor = conn.cursor()

# Only insert if table is empty (so you can re-run safely)
cursor.execute("SELECT COUNT(*) FROM departments")
(count,) = cursor.fetchone()

if count == 0:
    cursor.executemany(
        "INSERT INTO departments (dept_name) VALUES (%s)",
        [(d,) for d in departments]
    )
    conn.commit()
    print(" Inserted departments.")
else:
    print(f" Departments already exist (rows: {count}).")

cursor.execute("SELECT * FROM departments")
print("Current departments:")
for row in cursor.fetchall():
    print(row)

cursor.close()
conn.close()


 Inserted departments.
Current departments:
(1, 'IT')
(2, 'Finance')
(3, 'HR')
(4, 'Engineering')


In [4]:
##Insert Random users according to departments
NUM_USERS = 23  # you can change this

conn = get_connection()
cursor = conn.cursor()

# Fetch department IDs
cursor.execute("SELECT dept_id, dept_name FROM departments")
dept_rows = cursor.fetchall()
dept_ids = [row[0] for row in dept_rows]

print("Departments (id, name):", dept_rows)

# Check current users
cursor.execute("SELECT COUNT(*) FROM users")
(user_count,) = cursor.fetchone()

if user_count == 0:
    users_to_insert = []
    for _ in range(NUM_USERS):
        username = fake.user_name()
        dept_id = random.choice(dept_ids)
        users_to_insert.append((username, dept_id))

    cursor.executemany(
        "INSERT INTO users (username, dept_id) VALUES (%s, %s)",
        users_to_insert
    )
    conn.commit()
    print(f" Inserted {NUM_USERS} users.")
else:
    print(f" Users already exist (rows: {user_count}).")

# Preview a few users
cursor.execute("""
    SELECT u.user_id, u.username, d.dept_name
    FROM users u
    JOIN departments d ON u.dept_id = d.dept_id
    LIMIT 10;
""")
rows = cursor.fetchall()
print("\nSample users:")
for row in rows:
    print(row)

cursor.close()
conn.close()


Departments (id, name): [(1, 'IT'), (2, 'Finance'), (3, 'HR'), (4, 'Engineering')]
 Inserted 23 users.

Sample users:
(1, 'jperry', 'IT')
(9, 'kimsharp', 'IT')
(10, 'scottburnett', 'IT')
(13, 'richard69', 'IT')
(16, 'huberbrittany', 'IT')
(18, 'davissamantha', 'IT')
(20, 'richardcowan', 'IT')
(21, 'hortonamanda', 'IT')
(3, 'alexanderphillips', 'Finance')
(5, 'richardsmith', 'Finance')


In [5]:
##Generate log in events (For Dashboard)
NUM_DAYS = 30         # last 30 days
LOGINS_PER_DAY = 300  # how many login events per day

conn = get_connection()
cursor = conn.cursor()

# Get all user IDs
cursor.execute("SELECT user_id FROM users")
user_ids = [row[0] for row in cursor.fetchall()]

print(f"Users found: {len(user_ids)}")

start_date = datetime.now() - timedelta(days=NUM_DAYS)
login_rows = []

for day in range(NUM_DAYS):
    day_base = start_date + timedelta(days=day)
    for _ in range(LOGINS_PER_DAY):
        user_id = random.choice(user_ids)

        # Time: mostly normal office hours, sometimes suspicious off-hours
        if random.random() < 0.15:
            hour = random.choice([0, 1, 2, 3, 4])  # 15% unusual
        else:
            hour = random.randint(7, 22)
        minute = random.randint(0, 59)
        event_time = day_base.replace(hour=hour, minute=minute, second=0, microsecond=0)

        ip_address = fake.ipv4_public()
        location = random.choice(["India", "USA", "EU", "Unknown"])
        device = random.choice(["Windows", "Linux", "MacOS", "Mobile"])

        # 20% failed logins
        if random.random() < 0.2:
            is_success = 0
            failure_reason = random.choice(["WRONG_PASSWORD", "ACCOUNT_LOCKED", "UNKNOWN_DEVICE"])
        else:
            is_success = 1
            failure_reason = None

        login_rows.append(
            (user_id, event_time, ip_address, location, is_success, failure_reason, device)
        )

print(f"Prepared {len(login_rows)} login events.")

# Insert into DB
insert_sql = """
INSERT INTO login_events
(user_id, event_time, ip_address, location, is_success, failure_reason, device)
VALUES (%s, %s, %s, %s, %s, %s, %s);
"""

cursor.executemany(insert_sql, login_rows)
conn.commit()
print(" Inserted login_events rows.")

# Quick sanity check
cursor.execute("SELECT COUNT(*) FROM login_events")
(total_logins,) = cursor.fetchone()
print(f"Total login_events rows now: {total_logins}")

cursor.close()
conn.close()



Users found: 23
Prepared 9000 login events.
 Inserted login_events rows.
Total login_events rows now: 9000


In [6]:
##Generating vulnerabilities Data
from datetime import datetime, timedelta
import random

conn = get_connection()
cursor = conn.cursor()

# Get departments
cursor.execute("SELECT dept_id FROM departments")
dept_ids = [row[0] for row in cursor.fetchall()]
print("Departments:", dept_ids)

NUM_VULNS = 300   # change if you want
NUM_DAYS = 30

severities = ["LOW", "MEDIUM", "HIGH", "CRITICAL"]

start_date = datetime.now() - timedelta(days=NUM_DAYS)
vuln_rows = []

for _ in range(NUM_VULNS):
    machine_id = f"HOST-{random.randint(1000, 9999)}"
    dept_id = random.choice(dept_ids)

    detected_at = start_date + timedelta(
        days=random.randint(0, NUM_DAYS),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59),
    )

    # 60% chance remediated
    if random.random() < 0.6:
        remediated_at = detected_at + timedelta(days=random.randint(1, 10))
    else:
        remediated_at = None

    cve_id = f"CVE-{random.randint(2016, 2025)}-{random.randint(1000, 99999)}"
    severity = random.choices(
        severities,
        weights=[0.2, 0.4, 0.25, 0.15]  # more MEDIUM / HIGH
    )[0]

    vuln_rows.append(
        (machine_id, dept_id, cve_id, severity, detected_at, remediated_at)
    )

print("Prepared vulnerability rows:", len(vuln_rows))

insert_sql = """
INSERT INTO vulnerabilities
(machine_id, dept_id, cve_id, severity, detected_at, remediated_at)
VALUES (%s, %s, %s, %s, %s, %s)
"""

cursor.executemany(insert_sql, vuln_rows)
conn.commit()

cursor.execute("SELECT COUNT(*) FROM vulnerabilities")
(total_vulns,) = cursor.fetchone()
print(" Total vulnerabilities rows now:", total_vulns)

cursor.close()
conn.close()


Departments: [1, 2, 3, 4]
Prepared vulnerability rows: 300
 Total vulnerabilities rows now: 300


In [ ]:
##

In [7]:
import pandas as pd

conn = get_connection()

query = """
SELECT
    DATE(event_time) AS login_date,
    COUNT(*) AS total_logins,
    SUM(CASE WHEN is_success = 0 THEN 1 ELSE 0 END) AS failed_logins
FROM login_events
GROUP BY DATE(event_time)
ORDER BY login_date;
"""

df = pd.read_sql(query, conn)
conn.close()

df.head()


/var/folders/wb/1wq48jnx23lfy6szsc1fmz0h0000gn/T/ipykernel_1956/1671569185.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,login_date,total_logins,failed_logins
0,2025-10-11,300,59.0
1,2025-10-12,300,59.0
2,2025-10-13,300,68.0
3,2025-10-14,300,61.0
4,2025-10-15,300,48.0
